In [9]:
import pandas as pd
import numpy as np

schedule = pd.read_csv('input/schedule_5min_0612.csv')
schedule['schedule'] = np.ceil(schedule['schedule'] / 5) * 5
schedule




,schedule,od
0,5.0,LAX_DTLA
1,10.0,LAX_DTLA
2,15.0,LAX_DTLA
3,20.0,LAX_DTLA
4,25.0,LAX_DTLA
...,...,...
766,1415.0,DTLA_LAX
767,1425.0,DTLA_LAX
768,1430.0,DTLA_LAX
769,1435.0,DTLA_LAX


In [16]:
LAX_DTLA = schedule[schedule['od'] == 'LAX_DTLA'].reset_index()
LAX_DTLA['index'] = LAX_DTLA['index'].apply(lambda x: 'LAX_DTLA_' + str(x))
LAX_DTLA

,index,schedule,od
0,LAX_DTLA_0,5.0,LAX_DTLA
1,LAX_DTLA_1,10.0,LAX_DTLA
2,LAX_DTLA_2,15.0,LAX_DTLA
3,LAX_DTLA_3,20.0,LAX_DTLA
4,LAX_DTLA_4,25.0,LAX_DTLA
...,...,...,...
377,LAX_DTLA_377,1435.0,LAX_DTLA
378,LAX_DTLA_378,1440.0,LAX_DTLA
379,LAX_DTLA_379,1440.0,LAX_DTLA
380,LAX_DTLA_380,1440.0,LAX_DTLA


In [19]:
DTLA_LAX = schedule[schedule['od'] == 'DTLA_LAX'].reset_index()
DTLA_LAX['index'] = DTLA_LAX['index'].apply(lambda x: 'DTLA_LAX_' + str(x))
DTLA_LAX

,index,schedule,od
0,DTLA_LAX_382,10.0,DTLA_LAX
1,DTLA_LAX_383,15.0,DTLA_LAX
2,DTLA_LAX_384,25.0,DTLA_LAX
3,DTLA_LAX_385,45.0,DTLA_LAX
4,DTLA_LAX_386,55.0,DTLA_LAX
...,...,...,...
384,DTLA_LAX_766,1415.0,DTLA_LAX
385,DTLA_LAX_767,1425.0,DTLA_LAX
386,DTLA_LAX_768,1430.0,DTLA_LAX
387,DTLA_LAX_769,1435.0,DTLA_LAX


In [22]:
LAX_DTLA['arrival_time'] = LAX_DTLA['schedule'] + 15
DTLA_LAX['arrival_time'] = DTLA_LAX['schedule'] + 15

In [33]:
nodes = ['Source', 'Sink']
nodes.extend(LAX_DTLA['index']+'A')
nodes.extend(LAX_DTLA['index']+'B')
nodes.extend(DTLA_LAX['index']+'A')
nodes.extend(DTLA_LAX['index']+'B')

In [34]:
edges = [('Source', 'Sink')]
cost = [-100]
for i in LAX_DTLA['index']:
    edges.append(('Source', i+'A'))
    cost.append(1)
    edges.append((i+'A', i+'B'))
    cost.append(0)
    edges.append((i+'B', 'Sink'))
    cost.append(0)
for i in DTLA_LAX['index']:
    edges.append(('Source', i+'A'))
    cost.append(1)
    edges.append((i+'A', i+'B'))
    cost.append(0)
    edges.append((i+'B', 'Sink'))
    cost.append(0)

for previous_task in LAX_DTLA.iterrows():
    for next_task in LAX_DTLA.iterrows():
        if previous_task[1]['arrival_time'] <= next_task[1]['schedule']:
            edges.append((previous_task[1]['index']+'B', next_task[1]['index']+'A'))
            cost.append(0)

for previous_task in DTLA_LAX.iterrows():
    for next_task in DTLA_LAX.iterrows():
        if previous_task[1]['arrival_time'] <= next_task[1]['schedule']:
            edges.append((previous_task[1]['index']+'B', next_task[1]['index']+'A'))
            cost.append(0)

for previous_task in LAX_DTLA.iterrows():
    for next_task in DTLA_LAX.iterrows():
        if previous_task[1]['arrival_time']+15 <= next_task[1]['schedule']:
            edges.append((previous_task[1]['index']+'B', next_task[1]['index']+'A'))
            cost.append(0)

for previous_task in DTLA_LAX.iterrows():
    for next_task in LAX_DTLA.iterrows():
        if previous_task[1]['arrival_time']+15 <= next_task[1]['schedule']:
            edges.append((previous_task[1]['index']+'B', next_task[1]['index']+'A'))
            cost.append(0)

In [35]:
capacities = [GRB.INFINITY] + [1] * len(edges)
capacities = dict(zip(edges, capacities))

cost = dict(zip(edges, cost))
supply = {'Source': -20, 'Sink': 20}

In [38]:
from gurobipy import Model, GRB

# Initialize the model
m = Model("MCNF")

# Create variables for the flow on each edge
flow = m.addVars(edges, obj=cost, name="flow", lb=0, ub=capacities)

# Add supply/demand constraints for each node
for n in nodes:
    m.addConstr(sum(flow[i, j] for i, j in edges if j == n) -
                sum(flow[i, j] for i, j in edges if i == n) == supply.get(n, 0), f"node_{n}")

for i, j in edges:
    if i.endswith('A') and j.endswith('B'):
        m.addConstr(flow[i, j] == 1, f"flow_{i}_to_{j}")

# The objective is already set to minimize total cost during variable creation



# Optimize the model
m.optimize()

# Retrieve and print the solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    for i, j in edges:
        if solution[i, j] > 0:
            print(f"Flow from {i} to {j}: {solution[i, j]}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) Gold 6326 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [37]:
%env GRB_LICENSE_FILE=/home/albert/gurobipy/gurobi.lic


env: GRB_LICENSE_FILE=/home/albert/gurobipy/gurobi.lic
